In [ ]:
# Chris Jones - crj341@student.bham.ac.uk
# 28/04/2021

# Mass AFM analysis script:
# 1. Put all force curves to analyse in one folder
# 2. Set folder name below
# 3. Run 1st cell
# 3. Name each sample in the 'samples' variable below - note each name nust be in quotation marks, followed by ': [],'
# 4. PRun 2nd cell
# 5. Plot data below, calling the relevant samples using the sample names specified

import  glob, os
from    nanoforce               import  AFM
import  pandas                  as      pd
import  seaborn                 as      sns
import  plotly.graph_objects    as      go
from    plotly.subplots         import  make_subplots
import  numpy                   as      np
import  matplotlib.pyplot       as      plt

folder_name = r'C:\add_your_folder_path_here'

os.chdir(folder_name)
files = glob.glob("*.nid")
print(*files, sep='\n')

In [ ]:
# Analysis script - only change sample names!

samples = {
    'Add': [],
    'Your': [],
    'Sample': [],
    'Names': [],
    'Here': [],
}

i = 0

adhesion_df = pd.DataFrame()
modulus_df = pd.DataFrame()

for sample in samples.keys():

    samples[sample] = AFM()
    samples[sample].set_run_name(sample)
    samples[sample].file_name = files[i]

    i += 1

    samples[sample].nanosurf_import() # If using Nanoscope files replace this with:
                                      # samples[sample].input_files()
                                      # samples[sample].nanoscope_params()
                                      # samples[sample].nanoscope_read()
                                      
    samples[sample].baseline()
    samples[sample].contact()
    samples[sample].adjust_tip_position()

    samples[sample].calc_adhesion()
    samples[sample].calc_modulus()

    adhesion_df.insert(loc = 0, column=sample, value=samples[sample].adhesion)
    modulus_df.insert(loc = 0, column=sample, value=samples[sample].modulus)

In [ ]:
# Function to plot all force curves for specified sample
samples['Sample name'].plot_curves()

In [ ]:
# Function to overlay all approach and withdraw curves for specified sample
expt = samples['Sample name']

fig = make_subplots(rows=1, cols=2,
    subplot_titles=("Approach curves", "Retract curves"))
 
for curve_number in range(0,expt.approach.shape[1]):
 
    fig.add_trace(go.Scatter(y = expt.approach[:,curve_number], x = expt.z_approach[:,curve_number], mode='lines', name = 'Approach curve (raw)'), row=1, col=1)
    fig.add_trace(go.Scatter(y = expt.retract[:,curve_number], x = expt.z_withdraw[:,curve_number], mode='lines', name = 'Retract curve (raw)'), row=1, col=2)
 
fig['layout']['xaxis']['title']='Piezo position (nm)'
fig['layout']['xaxis2']['title']='Piezo position (nm)'
fig['layout']['yaxis']['title']='Force (nN)'
fig['layout']['yaxis2']['title']='Force (nN)'
 
fig.update_layout(height=500, width=1200, showlegend=False, template='plotly_white')
fig.show()

In [ ]:
# Adhesion histogram - see seaborn documentation to adjust formatting
ax1 = sns.histplot(adhesion_df[['Sample name 1','Sample name 2','Sample name 3']], kde=True)
ax1.set(xlabel='Adhesion (nN)', ylabel='Counts')

In [ ]:
# Adhesion bar chart - see seaborn documentation to adjust formatting
ax2 = sns.barplot(data = adhesion_df[['Sample name 1','Sample name 2','Sample name 3']])

In [ ]:
# Modulus histogram - see seaborn documentation to adjust formatting
ax3 = sns.histplot(modulus_df[['Sample name 1','Sample name 2','Sample name 3']], binwidth=0.25, kde=True)
ax3.set(xlabel='Adhesion (nN)', ylabel='Counts')